In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import h2o
from h2o.automl import H2OAutoML
import random
import os
import sys
from datetime import datetime
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

# Initialize H2O cluster
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Corretto-17.0.8.8.1 (build 17.0.8.1+8-LTS, mixed mode, sharing)
  Starting server from C:\Users\pchok\AppData\Local\Programs\Python\Python312\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\pchok\AppData\Local\Temp\tmprr6btrzi
  JVM stdout: C:\Users\pchok\AppData\Local\Temp\tmprr6btrzi\h2o_pchok_started_from_python.out
  JVM stderr: C:\Users\pchok\AppData\Local\Temp\tmprr6btrzi\h2o_pchok_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,3 months and 16 days
H2O_cluster_name:,H2O_from_python_pchok_pav74j
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.910 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [6]:
! pip install h2o
! pip install yellowbrick
! pip install requests
! pip install tabulate
! pip install "colorama>=0.3.8"
! pip install future
! pip install shap
! pip install datasist

   ---------------------------------------- 0.0/282.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/282.6 kB ? eta -:--:--
   ----- --------------------------------- 41.0/282.6 kB 667.8 kB/s eta 0:00:01
   --------------------------- ------------ 194.6/282.6 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 282.6/282.6 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/491.3 kB ? eta -:--:--
    --------------------------------------- 10.2/491.3 kB ? eta -:--:--
   --- ----------------------------------- 41.0/491.3 kB 653.6 kB/s eta 0:00:01
   ---------- ----------------------------- 122.9/491.3 kB 1.2 MB/s eta 0:00:01
   ------------------------- -------------- 317.4/491.3 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 491.3/491.3 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.6 kB 1.4 MB/s eta 0:00:01
  

In [10]:
#Reading data into pandas dataframe.

data_sets = "D:/INFO6105 Nick Brown/archive (2)/94_character_TMNIST.csv"
data = pd.read_csv(data_sets)

In [ ]:
# Displaying first rows of dataset
data.head()

,names,labels,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,Salsa-Regular,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MouseMemoirs-Regular,D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Creepster-Regular,f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SeoulNamsan-Light,/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HachiMaruPop-Regular,F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Drop redundant coloums (names and labels), retain only pixel values
X = data.drop(columns={'names','labels'})
y = data[['labels']] #Assigning Classes

In [12]:
import numpy as np

# Assuming X is your feature dataframe
X_reshaped = X.values.reshape(-1, 28, 28, 1).astype('float32') / 255.0


In [14]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode labels to integers
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y['labels'])

# One-hot encode the integer labels
y_one_hot = to_categorical(y_encoded)


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the data augmentation strategy
datagen = ImageDataGenerator(
    rotation_range=8,      # rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,# randomly shift images vertically (fraction of total height)
    zoom_range=0.2,        # randomly zoom image 
    shear_range=0.2,       # shear Intensity (Shear angle in counter-clockwise direction in degrees)
    fill_mode='nearest',   # points outside the boundaries of the input are filled according to the given mode
    validation_split=0.1   # Fraction of images reserved for validation
)

# Create training generator
train_generator = datagen.flow(
    X_reshaped,
    y_one_hot,
    batch_size=64,
    subset='training'  # Set as training data
)

# Create validation generator
validation_generator = datagen.flow(
    X_reshaped,
    y_one_hot,
    batch_size=64,
    subset='validation'  # Set as validation data
)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(y_one_hot.shape[1], activation='softmax')  # Assuming y_one_hot is your one-hot encoded labels
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\pchok\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [17]:
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)

callback = LearningRateScheduler(scheduler)


In [18]:
history = model.fit(
    train_generator,  # Assuming train_generator is set up for augmented training data
    epochs=60,
    validation_data=validation_generator,  # Assuming validation_generator is set up
    callbacks=[callback]
)


Epoch 1/60


c:\Users\pchok\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3855/3855 ━━━━━━━━━━━━━━━━━━━━ 103s 26ms/step - accuracy: 0.5086 - loss: 1.8275 - val_accuracy: 0.7577 - val_loss: 0.7728 - learning_rate: 0.0010
Epoch 2/60
3855/3855 ━━━━━━━━━━━━━━━━━━━━ 104s 27ms/step - accuracy: 0.7500 - loss: 0.7952 - val_accuracy: 0.8003 - val_loss: 0.6161 - learning_rate: 0.0010
Epoch 3/60
3855/3855 ━━━━━━━━━━━━━━━━━━━━ 103s 27ms/step - accuracy: 0.7778 - loss: 0.6939 - val_accuracy: 0.8117 - val_loss: 0.5846 - learning_rate: 0.0010
Epoch 4/60
3855/3855 ━━━━━━━━━━━━━━━━━━━━ 104s 27ms/step - accuracy: 0.7928 - loss: 0.6462 - val_accuracy: 0.8243 - val_loss: 0.5400 - learning_rate: 0.0010
Epoch 5/60
3855/3855 ━━━━━━━━━━━━━━━━━━━━ 107s 28ms/step - accuracy: 0.8046 - loss: 0.6109 - val_accuracy: 0.8258 - val_loss: 0.5327 - learning_rate: 0.0010
Epoch 6/60
3855/3855 ━━━━━━━━━━━━━━━━━━━━ 107s 28ms/step - accuracy: 0.8093 - loss: 0.5909 - val_accuracy: 0.8321 - val_loss: 0.5196 - learning_rate: 0.0010
Epoch 7/60
 111/3855 ━━━━━━━━━━━━━━━━━━━━ 1:33 25ms/step - accuracy: 